In [ ]:
import sys
sys.path.append('/home/ottok92/Dev/elogroup2/')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.eda import BCW_Explorer
from sklearn import preprocessing

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

sns.set()
sns.set_style("whitegrid", {'axes.grid' : False})

In [ ]:
eda = BCW_Explorer()

# 1. Data Quality

In [ ]:
eda.data.info()

# 2. Data Analysis

## 2.1 Overview of Variables

In [ ]:
print(
    'Attributes:\n\t{0}\n'.format(
        ', '.join(str(x) for x in eda.data.columns.get_level_values('attributes').unique() if x)
    ),
    '\n... where each attribute is summarized by:\n\t{0}'.format(
        ', '.join(str(x) for x in eda.data.columns.get_level_values('summaries').unique() if x != 'label')
    ),
    '\n\n... for a total of 30 features'
)


---
At first, we should assume that there is correlation between some of the features given the nature of the attributes which they summarize.
* radius yields (although the attribute is the mean of distances from center to points on the perimeter)
    * perimiter, area, smoothness (local variation in radius lengths) and compactness (perimeter^2 / area - 1.0);

We explore whether they significantly correlate in the section 2.3
<br />

\> **What is the attribute Fractal Dimension?**<br />
&emsp;Fractal dimension is a measure of how regular the contour of a shape is.<br />
It is approximated by measuring the downward slope of the log of observed perimeter plotted agains the log of a "ruler" size.<br />
Hence, a higher value corresponds to a less regular contour, which in turn means a higher probability of malignancy (Wolberg et al. 1994)


## 2.2 Research Questions (RQs)

1. What are the most significant features, and how to measure significance?
2. Which classifiers are most suitable for the task, and how to decide?
3. How much better (or worse) are classifiers induced from reduced dimensionality?
4. What performance measure should be favored during hyperparameter tuning?

## 2.3 Class Imbalance, Feature Distribution and Selection

In [ ]:
def plot_compare_classdistribution(in_df: pd.DataFrame, summary='mean', ax=None):
    # scales the data
    df = in_df.apply(preprocessing.scale, axis=0).copy()
    # df['label'] = df['label'].map({1: 'Malignant', 0: 'Benign'})

    # melts columns' (name, value) pair into two columns
    df =\
        pd.melt(
            pd.concat(
                [
                    df['label'],
                    df.xs(summary, level='summaries', axis=1)
                ],
                axis=1
            ),
            id_vars='label',
            var_name='features',
            value_name='value'
        )

    # plots comparative distribution of classes for each 'mean' feature
    return\
        sns.violinplot(
            ax=ax,
            data=df,
            y="value",
            x="features",
            hue="label",
            inner="quartile",
            split=True,
        )

from scipy.stats import boxcox, skew


def transform_skew(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Transforms data with Boxcox transformation
    Keeps the transformation only if it is less skewed than the original
    '''
    skew_transformed = pd.DataFrame()

    for feat in df.drop('label', axis=1):

        data = df.drop('label', axis=1)[feat].values
        posdata = data[data > 0]
        # posdata = eda.data[eda.data[feat]>0][feat]

        x, lmbda = boxcox(posdata, lmbda=None)
        
        transform = np.empty_like(data)
        transform[data > 0] = x
        transform[data == 0] = -1/lmbda

        if abs(skew(transform)) < abs(skew(data)):
            skew_transformed[feat] = transform

        else:
            skew_transformed[feat] = data
    
    skew_transformed.columns =\
        pd.MultiIndex.from_tuples(
            list(skew_transformed.columns),
            names=('summaries', 'attributes')
        )

    return skew_transformed

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.set_title('Balance of Classes', fontsize=14)

eda.data['label']\
    .map({1: 'Malignant', 0: 'Benign'})\
    .value_counts()\
    .plot(
        ax=ax,
        kind='pie',
        autopct='%1.1f%%',
        startangle=270,
        fontsize=14,
        label=''
    )

plt.show()

### - Imbalanced Classes
&emsp;There are almost twice as many benign observations than malignant, which can become an issue while inducing machine learning models. Thus, we must employ a sampling technique.<br />
<br />

<center><img style="margin-bottom:5mm" src="https://raw.githubusercontent.com/rafjaa/machine_learning_fecib/master/src/static/img/resampling.png"><center/>
<center>Figure 1. Approaches to resampling<center/>
<center style="margin-bottom:5mm">Source. https://tinyurl.com/yy2qtbe2<center/>

<p>&emsp;However, before applying any resampling method to our data, we must ensure that it meets some assumptions that sampling methods require; thus, we investigate the distribution of our features and whether there are any redundant candidates for removal. From both tasks, the former is comprised of statistical analysis and data visualization, while the latter of feature selection techniques (e.g. correlation analysis and recursive feature elimination (RFE))<p/>

### - Feature Distribution

In [ ]:
# -------------------------------------------------------------------------------
fig, ax = plt.subplots(1, 1, figsize=(20, 6))

ax.set_title('Comparative Distribution of Features (standardized)', fontsize=14)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plot_compare_classdistribution(eda.data, ax=ax, summary='mean')

ax.tick_params(axis='x', labelrotation=30)
ax.legend_.remove()
leg = ax.legend(['Quartiles', 'Mean'], loc='upper right', bbox_to_anchor=(1.25, 1))

plt.show()

In [ ]:
print('Skewness of features:')
eda.data.skew().sort_values(ascending=False).sort_index(level='summaries', sort_remaining=False)

In [ ]:
# applies boxcox transformation and decides whether to use it against the original variable
transformed = transform_skew(eda.data)
transformed['label'] = eda.data['label']

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 6))

ax.set_title('Comparative Distribution of Features (skewed transformed, standardized)', fontsize=14)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plot_compare_classdistribution(transformed)

ax.tick_params(axis='x', labelrotation=30)
ax.legend_.remove()
ax.legend(['Quartiles', 'Mean'], loc='upper right', bbox_to_anchor=(1.25, 1))

plt.show()

In [ ]:
print('Skewness after Boxcox transform:')
transformed.skew().sort_values(ascending=False).sort_index(level='summaries', sort_remaining=False)

### - Correlation Analysis

In [ ]:
def get_unique_correlation(data: pd.DataFrame, threshold=0.5) -> pd.DataFrame:
    try:
        tmp = data.drop('label', axis=1).copy()
    
    except KeyError:
        pass

    tmp.columns = tmp.columns.to_flat_index()
    dataCorr = tmp.corr(method='pearson')

    dataCorr = dataCorr.mask(np.tril(np.ones(dataCorr.shape)).astype(np.bool))
    dataCorr = dataCorr[abs(dataCorr) >= threshold].stack()

    dataCorr = dataCorr.reset_index(level=0)
    dataCorr.index = pd.MultiIndex.from_tuples(dataCorr.index, names=('summaries', 'attributes'))
    
    return dataCorr.sort_index(level='attributes').sort_index(level='summaries', sort_remaining=False)

In [ ]:
# -------------------------------------------------------------------------------
fig, ax = plt.subplots(1, 1, figsize=(17, 12))
ax.set_title('Correlation Matrix', fontsize=14)

corr = eda.data.drop('label', axis=1).corr()

# generates an upper diagonal mask
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(
    corr,
    ax=ax,
    vmax=1,
    cmap=cmap,
    mask=mask,
    annot=False,
    square=False,
)

plt.show()

**Feature Selection Stategy**<br />
&emsp;Before applying an automated search for the optimal dimensionality for each classifer, we investigate features we consider relevant for the sake of comparison. There are a few features (variables) that correlate with many others, which could lead to using them to represent all others. We must how these "subfeatures" correlated to one another. At the end of this process, we will be able to answer RQ1.
* RQ1 - What are the most significant features, and how to measure significance?

In [ ]:
# # selects features that have highest number of correlations above the given threshold
# above_threshold = corr.apply(lambda x: x > .9).sum()
# most_correlations = corr[above_threshold.apply(lambda x: x == above_threshold.max())]
# most_correlations[most_correlations.apply(lambda x: x > 0.9)].dropna(axis=1)

**Correlation clustering** is the task of ...

In [ ]:
def correlation_clustering(corr: np.ndarray, labels: np.ndarray, threshold=0.5, ax=plt.axes, maxclust=None) -> (dict or np.ndarray):
    '''
    '''
    dissimilarity = 1 - np.abs(corr)
    hierarchy = linkage(squareform(dissimilarity), method='ward', metric='distance')
    
    return\
        (
            fcluster(hierarchy, maxclust, criterion='maxclust') if maxclust
            else
            dendrogram(
                hierarchy, color_threshold=None,
                labels=labels, ax=ax,
                distance_sort='ascending',
                leaf_font_size=12,
                orientation='right',
            )
        )
        

In [ ]:
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

fig, ax = plt.subplots(1, 1, figsize=(20, 12))
ax.set_title('Correlation Clustering', fontsize=14)

d = correlation_clustering(corr.values, corr.index.values, ax=ax)

# plt.xticks(rotation=90)
plt.show()


In [ ]:
# groups features into their respective clusters
cluster = correlation_clustering(corr.values, corr.index.values, maxclust=3)
cluster =\
    pd.Series(cluster, corr.index.values, name='cluster')\
        .reset_index().groupby('cluster')['index'].apply(np.array)\
        .to_dict()

In [ ]:
eda.data[cluster[1][0]]

In [ ]:
import plotly.graph_objects as go
from IPython.display import display
from plotly.subplots import make_subplots
from ipywidgets import interact, interact_manual

@interact
def viz_cluster_separation(
    feature1=[pair[0] +'_'+ pair[1] for pair in cluster[1]],
    feature2=[pair[0] +'_'+ pair[1] for pair in cluster[2]],
    feature3=[pair[0] +'_'+ pair[1] for pair in cluster[3]]
):
    '''
    '''
    labels = eda.data[('label', '')]
    feature1 = tuple(feature1.split('_', 1))
    feature2 = tuple(feature2.split('_', 1))
    feature3 = tuple(feature3.split('_', 1))

    fig =\
        make_subplots(
            rows=1, cols=2,
            specs=[
                [{'is_3d': True}, {'is_3d': True}]
            ],
            print_grid=False,
            subplot_titles=('Original features', 'Standardized features')
        )

    x = eda.data[feature1]
    y = eda.data[feature2]
    z = eda.data[feature3]

    fig.add_trace(
        go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='markers',
            showlegend=False,
            
            marker=dict(
                size=8,
                color=labels,
                colorscale=['#CC8963', '#5875A4'],
                opacity=1.0
            )
        ),
        row=1, col=1
    )

    x = transformed[feature1]
    y = transformed[feature2]
    z = transformed[feature3]

    fig.add_trace(
        go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='markers',
            showlegend=False,
            marker=dict(
                size=8,
                color=labels,
                colorscale=['#CC8963', '#5875A4'],
                opacity=1.0
            )
        ),
        row=1, col=2
    )

    fig.update_layout(
        width=1200,
        height=600,
        margin=dict(r=10, l=10, b=10, t=10),
        title_text='Class separation',
        scene = dict(
            xaxis_title='Mean Texture',
            yaxis_title='Worst Area',
            zaxis_title='Worst Smoothness'
        )
    )

    fig.show()


**Validate** clusters with Recursive Feature Elimination

In [ ]:
from sklearn.model_selection import train_test_split

std_X_train, std_X_test, std_y_train, std_y_test =\
    train_test_split(
        transformed.drop(('label', ''), axis=1).values,
        transformed[('label', '')],
        test_size=.2,
        random_state=42
    )

X_train, X_test, y_train, y_test =\
    train_test_split(
        eda.data.drop(('label', ''), axis=1).values,
        eda.data[('label', '')],
        test_size=.2,
        random_state=42
    )

In [ ]:
# from sklearn.feature_selection import RFE
# from sklearn.svm import SVR

# estimator = SVR(kernel="linear")
# selector = RFE(estimator, 3, step=1)
# selector = selector.fit(X_train, y_train)
# transformed.drop(('label', ''), axis=1).columns.to_flat_index()[selector.support_]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, chi2

pipe = Pipeline([
    # the reduce_dim stage is populated by the param_grid
    ('reduce_dim', 'passthrough'),
    ('classify', LinearSVC(dual=False, max_iter=10000))
])

N_FEATURES_OPTIONS = [3, 5, 8, 15]
C_OPTIONS = [1, 10, 100, 1000]
param_grid = [
    {
        'reduce_dim': [PCA(iterated_power=7), NMF()],
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__C': C_OPTIONS
    },
    {
        'reduce_dim': [SelectKBest(chi2)],
        'reduce_dim__k': N_FEATURES_OPTIONS,
        'classify__C': C_OPTIONS
    },
]
reducer_labels = ['PCA', 'NMF', 'KBest(chi2)']

grid = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid)
grid.fit(X, y)

mean_scores = np.array(grid.cv_results_['mean_test_score'])
# scores are in the order of param_grid iteration, which is alphabetical
mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# select score for best C
mean_scores = mean_scores.max(axis=0)
bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
               (len(reducer_labels) + 1) + .5)

plt.figure()
COLORS = 'bgrcmyk'
for i, (label, reducer_scores) in enumerate(zip(reducer_labels, mean_scores)):
    plt.bar(bar_offsets + i, reducer_scores, label=label, color=COLORS[i])

plt.title("Comparing feature reduction techniques")
plt.xlabel('Reduced number of features')
plt.xticks(bar_offsets + len(reducer_labels) / 2, N_FEATURES_OPTIONS)
plt.ylabel('Digit classification accuracy')
plt.ylim((0, 1))
plt.legend(loc='upper left')

plt.show()